# Fisher Test

(Continue from part 9 after exporting from Python)
NOTE: This is an R Jupyter notebook.

In [1]:
getwd()

[1] "/Users/ac4294/dev/2024/cpvt_database_analysis"

In [2]:
# load yaml
library(yaml)

# load config
config <- yaml::read_yaml(file.path(getwd(), ".", "config.yaml"))

version <- config$version

# convert to replace . with _
VERSION <- gsub("\\.", "_", version)

VERSION

[1] "0_3_4"

In [3]:
data_dir <- file.path(getwd(), ".", "figures", VERSION, "analysis_16_treatment_efficacy")

if (!dir.exists(data_dir)) {
  # raise error if the directory does not exist
  stmt <- paste0("The directory ", data_dir, " does not exist. Please run the analysis first.")
  print(stmt)
  stop(stmt)
}

output_dir <- file.path(getwd(), ".", "figures", VERSION, "analysis_16_treatment_efficacy")

if (!dir.exists(output_dir)) {
  dir.create(output_dir)

  print(paste("Created directory", output_dir))
}

In [4]:
library(rstatix)



Attaching package: ‘rstatix’


The following object is masked from ‘package:stats’:

    filter




In [5]:
p_adj_method <- "fdr"

In [6]:
# load the csv
fisher_flec <- read.csv(file.path(data_dir, "fisher_table_Flecainide.csv"))
fisher_flec_test <- fisher_flec[, c("p_hgvs_string",
                                    "is_effective..Flecainide.",
                                    "not_effective..Flecainide.")]
rownames(fisher_flec_test) <- fisher_flec_test$p_hgvs_string
fisher_flec_test <- fisher_flec_test[, -1]
fisher_flec_test

,is_effective..Flecainide.,not_effective..Flecainide.
,<int>,<int>
C2277R,6,0
R420W,6,1
S2246L,2,1


In [7]:
pairwise_results <- function(df_input, csv_name) {
  fisher_pair_results = pairwise_fisher_test(x = df_input, p.adjust.method = p_adj_method)
  # convert to df
  fisher_pair_results <- as.data.frame(fisher_pair_results)

  # order by p.value
  fisher_pair_results <- fisher_pair_results[order(fisher_pair_results$p.adj),]

  # remove scientific notation
  options(scipen = 999)

  # save to csv
  write.csv(fisher_pair_results, file.path(output_dir, csv_name))

  # return only significant results
  return(fisher_pair_results[fisher_pair_results$p.adj < 0.05,])
}


pairwise_results(
    fisher_flec_test,
    "results_flecainide.csv"
)

group1,group2,n,p,p.adj,p.adj.signif
<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>


In [8]:
# ICD
load_data <- function(
    file_name,
    treatment_name
) {
    fish_table <- read.csv(
        file.path(
            data_dir,
            file_name
        )
    )

    print(colnames(fish_table))
    fish_table <- fish_table[, c(
        "p_hgvs_string",
        paste0("is_effective..", treatment_name, "."),
        paste0("not_effective..", treatment_name, ".")
    )]
    rownames(fish_table) <- fish_table$p_hgvs_string
    fish_table <- fish_table[, -1]
    return(fish_table)
}

# fisher_icd <- read.csv(file.path(data_dir, "fisher_table_Implantable cardioverter-defibrillator.csv"))
# fisher_icd_test <- fisher_icd[, c("p_hgvs_string",
#                                   "ICD_and_beta_blocker",
#                                   "beta_blocker_only")]
# rownames(fisher_icd_test) <- fisher_icd_test$p_hgvs_string
# fisher_icd_test <- fisher_icd_test[, -1]
fisher_icd_test <- load_data(
    "fisher_table_Implantable cardioverter-defibrillator.csv",
    "Implantable.cardioverter.defibrillator"
)
fisher_icd_test

[1] "p_hgvs_string"                                         
[2] "is_effective..Implantable.cardioverter.defibrillator." 
[3] "not_effective..Implantable.cardioverter.defibrillator."
[4] "Total"                                                 


,is_effective..Implantable.cardioverter.defibrillator.,not_effective..Implantable.cardioverter.defibrillator.
,<int>,<int>
R420Q,4,5
R420W,5,1
S2246L,6,2


In [9]:


pairwise_results(
    fisher_icd_test,
    "results_icd.csv"
)

group1,group2,n,p,p.adj,p.adj.signif
<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>


In [10]:
fisher_beta_blocker <- load_data(
    "fisher_table_Beta blocker.csv",
    "Beta.blocker"
)
fisher_beta_blocker

[1] "p_hgvs_string"                "is_effective..Beta.blocker." 
[3] "not_effective..Beta.blocker." "Total"                       


,is_effective..Beta.blocker.,not_effective..Beta.blocker.
,<int>,<int>
C2277R,2,6
G2337V,10,1
G357S,3,0
R420Q,10,8
R420W,5,12
S2246L,4,5


In [11]:
pairwise_results(fisher_beta_blocker, "results_bb.csv")

,group1,group2,n,p,p.adj,p.adj.signif
,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>
8,G2337V,R420W,28,0.00208,0.0312,*
1,C2277R,G2337V,19,0.00627,0.0470,*
